In [1]:
import os, importlib, sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# Common paths
BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULE_PATH = os.path.join(BASE_PATH, "modules")
DS_PATH = os.path.join(BASE_PATH, "datasets")

In [3]:
sys.path.append(MODULE_PATH)

import mp.MomentPropagation as mp
importlib.reload(mp)

import data.mnist as mnist_loader
importlib.reload(mnist_loader)

<module 'data.mnist' from '/home/exleonem/Desktop/workspace/thesis/wp/notebooks/model/../../modules/data/mnist.py'>

In [4]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import  Conv2D, MaxPool2D, Dropout, Flatten, Dense, Softmax

In [5]:
gpus = tf.config.experimental.list_physical_devices("GPU")
cpus = tf.config.experimental.list_physical_devices("CPU")

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPU's, ", len(logical_gpus), "Logical GPU's")
        
    except RuntimeError as e:
        print(e)
        
elif cpus:
    try:
        logical_cpus = tf.config.experimental.list_logical_devices("CPU")
        print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPU")
        
    except RuntimeError as e:
        print(e)
        
        
tfk = tf.keras

1 Physical GPU's,  1 Logical GPU's


In [6]:
input_shape = (28, 28, 1)
model = Sequential([
    Conv2D(128, 4,activation="relu", input_shape=input_shape),
    MaxPool2D(),
    Dropout(.2),
    Conv2D(64, 3, activation="relu"),
    MaxPool2D(),
    Dropout(.2),
    Flatten(),
    Dense(512, activation="relu"),
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.build(input_shape)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 25, 128)       2176      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0

In [7]:
import active_learning as active
importlib.reload(active)

<module 'active_learning' from '/home/exleonem/Desktop/workspace/thesis/wp/notebooks/model/../../modules/active_learning/__init__.py'>

In [8]:
new_checkpoint = active.Checkpoint("mc_dropout")
new_checkpoint.load(model)

In [9]:
import mp.MomentPropagation as mp
importlib.reload(mp)

mp = mp.MP()
mp_model = mp.create_MP_Model(model=model, use_mp=True, verbose=True)

In [10]:
import data.mnist as mnist_loader
importlib.reload(mnist_loader)

# Load Data
mnist_path = os.path.join(DS_PATH, "mnist")
inputs, targets = mnist_loader.load(mnist_path)

# Select only first and second class
selector = (targets==0) | (targets==1)
new_inputs = inputs[selector].astype("float32")/255.0
new_targets = targets[selector]

# Create splits
x_train, x_test, y_train, y_test = train_test_split(new_inputs, new_targets)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test)

In [15]:
new_inputs.shape

(14780, 28, 28)

In [23]:
new_inputs[:, None, ...].shape

(14780, 1, 28, 28)

In [26]:
sample_input = np.expand_dims(new_inputs, axis=-1)
sample_input.shape

(14780, 28, 28, 1)

In [28]:
model.predict(sample_input[:5])

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [13]:
%load_ext autoreload 
%autoreload 2

import acl
importlib.reload(acl)
from acl import ActiveLearning

import active_learning
importlib.reload(active_learning)
from active_learning import TrainConfig, Config

train_config = TrainConfig(
    batch_size=2,
    epochs=1
)

acq_config = Config(
    name="std_mean",
    pseudo=True
)


active_learning = ActiveLearning(
    model, 
    np.expand_dims(new_inputs, axis=-1), labels=new_targets, 
    train_config=train_config
)

active_learning.start(step_size=400)

Training time: 12.79493522644043//Acquisition: 0.001264333724975586//Update: 2.7894973754882812e-05: 100%|██████████| 37/37 [05:12<00:00,  8.46s/it]  


[{'iteration': 0, 'training': None},
 {'iteration': 400,
  'training': {'loss': [5.092104692872874e-34], 'accuracy': [1.0]}},
 {'iteration': 800,
  'training': {'loss': [2.3962803612285135e-34], 'accuracy': [1.0]}},
 {'iteration': 1200,
  'training': {'loss': [4.622803817999438e-33], 'accuracy': [1.0]}},
 {'iteration': 1600,
  'training': {'loss': [7.218652952037088e-34], 'accuracy': [1.0]}},
 {'iteration': 2000,
  'training': {'loss': [1.0226904442593568e-32], 'accuracy': [1.0]}},
 {'iteration': 2400,
  'training': {'loss': [5.896790636425217e-33], 'accuracy': [1.0]}},
 {'iteration': 2800,
  'training': {'loss': [7.489674011207713e-33], 'accuracy': [1.0]}},
 {'iteration': 3200,
  'training': {'loss': [3.621315765345863e-32], 'accuracy': [1.0]}},
 {'iteration': 3600,
  'training': {'loss': [8.381284713064892e-32], 'accuracy': [1.0]}},
 {'iteration': 4000,
  'training': {'loss': [1.2655430955670288e-31], 'accuracy': [1.0]}},
 {'iteration': 4400,
  'training': {'loss': [2.371942476195157